In [64]:
import requests
%pip install streamlit
import pandas as pd
import streamlit as st

%pip install altair pandas vega_datasets
%pip install -U altair_viewer
%pip install streamlit
import altair as alt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [44]:
jsonlist = requests.get("https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=state:*").json()
states =[]
populations =[]
for list in jsonlist:
    states.append(list[0])
    populations.append(list[1])
data = {"State":states,"Population":populations
}
df=pd.DataFrame(data)
df = df.iloc[1:]
df["Population"]=pd.to_numeric(df["Population"])
df.head()





,State,Population
1,Mississippi,2976149
2,Missouri,6137428
3,Montana,1068778
4,Nebraska,1934408
5,Nevada,3080156


In [45]:
chart = alt.Chart(df).mark_area().encode(
    x="State",
    y="Population"
)
st.title('Population Chart')
st.altair_chart(
    chart, use_container_width=True)


/Users/maddonk/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


DeltaGenerator()

Vega-altair orders state IDs in aphabetical order so I ordered the data frame by state name

In [46]:
from vega_datasets import data
df = df.sort_values(by="State")
df=df.reset_index(drop=True)
#df['id']= data.population_engineers_hurricanes()
df.head()
## just a random vega dataset that uses same id convention for states
altair_state_ids = data.population_engineers_hurricanes()
df["id"]=altair_state_ids['id']
df.head()


,State,Population,id
0,Alabama,4903185,1
1,Alaska,731545,2
2,Arizona,7278717,4
3,Arkansas,3017804,5
4,California,39512223,6


In [47]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')


alt.Chart(states).mark_geoshape().encode(
    color='Population:Q',
    tooltip=['State:N', 'Population:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'id', ['Population','State'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)


alt.Chart(...)

In [48]:
import altair as alt
from vega_datasets import data

# Load US states TopoJSON data
states = alt.topo_feature(data.us_10m.url, 'states')
# Display the first feature's properties
alt.Chart(states).mark_geoshape().transform_calculate(
    properties=alt.datum.properties
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.Chart(...)

List of state names and state codes in corresponding order

In [49]:
state_codes = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

state_names = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California",
    "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas",
    "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts",
    "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana",
    "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma",
    "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]

In [87]:
from scrapy import Selector
medians =[]

for state in state_codes:
   
    populationurl= 'https://www.census.gov/quickfacts/fact/table/'+state+'/INC110221#INC110221'
   
    response = requests.get(populationurl)
    sel= Selector(text=response.text)
    medians.append(sel.css('td::text').getall()[3])

medians

['$54,943',
 '$80,287',
 '$65,913',
 '$52,123',
 '$84,097',
 '$80,184',
 '$83,572',
 '$72,724',
 '$61,777',
 '$65,030',
 '$88,005',
 '$63,377',
 '$72,563',
 '$61,944',
 '$65,429',
 '$64,521',
 '$55,454',
 '$53,571',
 '$63,182',
 '$91,431',
 '$89,026',
 '$63,202',
 '$77,706',
 '$49,111',
 '$61,043',
 '$60,560',
 '$66,644',
 '$65,686',
 '$83,449',
 '$89,703',
 '$54,020',
 '$75,157',
 '$60,516',
 '$68,131',
 '$61,938',
 '$56,956',
 '$70,084',
 '$67,587',
 '$74,489',
 '$58,234',
 '$63,920',
 '$58,516',
 '$67,321',
 '$79,133',
 '$67,674',
 '$80,615',
 '$82,400',
 '$50,884',
 '$67,080',
 '$68,002']

In [88]:
df2 = pd.DataFrame({"State":state_names,"Median Household income":medians})
df2

,State,Median Household income
0,Alabama,"$54,943"
1,Alaska,"$80,287"
2,Arizona,"$65,913"
3,Arkansas,"$52,123"
4,California,"$84,097"
5,Colorado,"$80,184"
6,Connecticut,"$83,572"
7,Delaware,"$72,724"
8,Florida,"$61,777"
9,Georgia,"$65,030"


In [89]:
merged_df = pd.merge(df,df2)
merged_df.head()

,State,Population,id,Median Household income
0,Alabama,4903185,1,"$54,943"
1,Alaska,731545,2,"$80,287"
2,Arizona,7278717,4,"$65,913"
3,Arkansas,3017804,5,"$52,123"
4,California,39512223,6,"$84,097"


In [90]:
url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area'
response = requests.get(url)
df3


,State,Land mi2
0,Alaska,570641
1,Texas,261232
2,California,155779
3,Montana,145546
4,New Mexico,121298
5,Arizona,113594
6,Nevada,109781
7,Colorado,103642
8,Oregon,95988
9,Wyoming,97093


In [91]:
htmlresponse = pd.read_html(url,header=0)
df3 = htmlresponse[0]
df3 = df3[['State / territory','Land mi2']]
df3 = df3.head(51)
df3 = df3.rename(columns={'State / territory': 'State'})
df3.head()


,State,Land mi2
0,Alaska,570641
1,Texas,261232
2,California,155779
3,Montana,145546
4,New Mexico,121298


In [92]:
merged_df = pd.merge(merged_df,df3)


merged_df.head()

,State,Population,id,Median Household income,Land mi2
0,Alabama,4903185,1,"$54,943",50645
1,Alaska,731545,2,"$80,287",570641
2,Arizona,7278717,4,"$65,913",113594
3,Arkansas,3017804,5,"$52,123",52035
4,California,39512223,6,"$84,097",155779


In [93]:
merged_df["Population/SqMi"]= merged_df['Population'] / merged_df['Land mi2']
merged_df

,State,Population,id,Median Household income,Land mi2,Population/SqMi
0,Alabama,4903185,1,"$54,943",50645,96.814789
1,Alaska,731545,2,"$80,287",570641,1.281971
2,Arizona,7278717,4,"$65,913",113594,64.076597
3,Arkansas,3017804,5,"$52,123",52035,57.995657
4,California,39512223,6,"$84,097",155779,253.642808
5,Colorado,5758736,8,"$80,184",103642,55.563729
6,Connecticut,3565287,9,"$83,572",4842,736.325279
7,Delaware,973764,10,"$72,724",1949,499.622370
8,Florida,21477737,12,"$61,777",53625,400.517240
9,Georgia,10617423,13,"$65,030",57513,184.609097


creating integer value for median income

In [98]:
merged_df['Income_Integer'] = merged_df['Median Household income'].replace('[\$,]', '', regex=True).astype(int)
merged_df['Population/SqMi'] = pd.to_numeric(merged_df['Population/SqMi'], errors='coerce')
scatter = alt.Chart(merged_df).mark_circle().encode(
    y='Income_Integer',
    x='Population/SqMi'
)
regressionline = scatter.transform_regression('Income_Integer', 'Population/SqMi').mark_line()
combined_chart = (scatter + regressionline)
combined_chart.interactive()


alt.LayerChart(...)

In [99]:
scatter.interactive()

alt.Chart(...)